# Chaining Functions

Running one prompt at a time can produce fantastic results! Sometimes you need several outputs from your AI copilot though. Of course you can simply call different skills in sequence in what is known as a prompt chain.

The simplest version of a prompt chain would be implemented by calling InvokeAsync for each function in the order you wanted them to execute.

A more interesting version of chaining allows you to use the output of one function to feed the next function thereby modifying the final result. Let's take a look at one of these more complex prompt chains.

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.17.230626.1-preview"

#!import ../config/SettingsHelper.cs

using Microsoft.SemanticKernel;

// load settings specific to you!
MySettings settings = Settings.LoadFromFile();

// Configure AI backend used by the kernel
var builder = new KernelBuilder();
if (settings.Type == "azure")
    builder.WithAzureTextCompletionService(settings.AzureOpenAI.CompletionsDeployment, settings.AzureOpenAI.Endpoint, settings.AzureOpenAI.ApiKey);
else
    builder.WithOpenAITextCompletionService(settings.OpenAI.Model, settings.OpenAI.ApiKey, settings.OpenAI.OrgId);
IKernel kernel = builder.Build();

The IKernel.RunAsync function allows you to provide one or more variables using a ContextVariables object and then any number of Semantic Kernel Functions in the order you want them to execute.

The functions can be any combination of any supported types of functions including semantic, inline, or native functions. Only native functions allow you to modify the ContextVariables object though. This is an important capability in many prompt chaining solutions.

Run the following example.

In [ ]:
using Microsoft.SemanticKernel.Orchestration;
using System.IO;

#!import Plugins/ChaosPlugin.cs
#!import Plugins/MadLibPlugin.cs

// you can use any combination of semantic, native, or inline functions in a chain
var pluginsDirectory = Path.Combine(Directory.GetCurrentDirectory(), "Plugins");
var puzzlePlugin = kernel.ImportSemanticSkillFromDirectory(pluginsDirectory, "PuzzlePlugin");

var madLibPlugin = kernel.ImportSkill(new MadLibPlugin(kernel));
var chaosPlugin = kernel.ImportSkill(new ChaosPlugin(kernel));

// define your desired parameters needed by the chain of functions.
// some variables will be populated by the native functions in the chain as well!
ContextVariables pipelineContext = new();
pipelineContext["madLibTheme"] = "rock concert";

// pass your prompt to your OpenAI instance and retrieve the completion
var completion = await kernel.RunAsync(
    pipelineContext, 
    chaosPlugin["GenerateRandomNumbers"], 
    chaosPlugin["GenerateRandomWords"], 
    madLibPlugin["GenerateMadLib"], 
    puzzlePlugin["FillTheBlanksFunction"]);

Console.WriteLine(completion);

There's actually quite a bit of code in this solution and it's spread across a few different files, but we'll walk through it here as best we can.

First, we'll load all of the plugins we'll need.
- The Puzzle Plugin has just one semantic function defined. It's job is to take a list of random adjectives, nouns, and verbs.

# Exercise

Create a new prompt chain solution targeting international students learning English. You want to surprise them every day with a list of words and their definitions.

**Hint: Use the Chaos Plugin's Generate Random Numbers and Generate Random Words functions. You'll have to write your own semantic, inline, or native function to fetch the definition of words.**